# LOADING DATA & PRETRAINED MODEL

In [37]:
from datasets import load_dataset
from transformers import pipeline
import math
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

device = 'cpu'
model_ckpt = 'facebook/bart-large'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

dataset = load_dataset('csv', data_files={'train': "./data/train.csv", 'test': "./data/test.csv", 'validate': "./data/validation.csv"})

dataset['validate'] = dataset['validate'].select(range(5000))

def filter_empty_rows(example):
    return all(value for value in example.values())

# Filter the dataset using the custom filtering function
dataset = dataset.filter(filter_empty_rows)

In [39]:
#Data Collator.

def get_feature(batch):
  """
  This collarate the content of the inputs to the abstract of the result using the formatted encodings.
  """
  encodings = tokenizer(batch['Content'], text_target=batch['Abstract'],
                        max_length=1024, truncation=True)

  encodings = {'input_ids': encodings['input_ids'],
               'attention_mask': encodings['attention_mask'],
               'labels': encodings['labels']}

  return encodings

In [ ]:
data = dataset.map(get_feature, batched=True)

In [41]:
columns = ['input_ids', 'labels', 'attention_mask']
data.set_format(type='torch', columns=columns)

# FINETUNING 

In [42]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [43]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = './model/bart_vietnews_full',
    num_train_epochs=1,                         # Number of training epochs. Here, it's set to 1. (>1 leads to longer training time)
    warmup_steps = 500,                         # Number of steps for the learning rate warmup.
    per_device_train_batch_size=4,              # Batch size per GPU/TPU core/CPU for training.
    per_device_eval_batch_size=4,               # Batch size per GPU/TPU core/CPU for evaluation.
    weight_decay = 0.01,                        # Weight decay for regularization to prevent overfitting.
    logging_steps = 10,                         # Log training information every 10 steps.
    evaluation_strategy = 'steps',              # Evaluation strategy to use: 'steps' (evaluation occurs at regular intervals.)
    eval_steps=500,                             # Number of update steps between evaluations.
    save_steps=1e6,                             # Number of update steps before saving the model. 
    gradient_accumulation_steps=16              # Number of update steps to accumulate the gradients before performing a backward/update pass.
)

trainer = Trainer(model=model, 
                  args=training_args, 
                  tokenizer=tokenizer, 
                  data_collator=data_collator,          
                  train_dataset = data['train'], 
                  eval_dataset = data['test'])
#

In [ ]:
trainer.train()
trainer.save_model('./model/bart_vietnews_model')

# EXAMPLE USAGE

In [ ]:
from datasets import load_dataset
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

In [ ]:

pipe = pipeline('summarization', model='./model/bart_vietnews_model')

custom_dialogue="""
Ngày 7-4, một lãnh đạo Công an TP Hải Phòng cho biết Phòng cảnh sát điều tra tội phạm về ma túy Công an TP vừa triệt phá ổ nhóm "bay lắc" tại một khu đô thị cao cấp trên địa bàn phường Thượng Lý, quận Hồng Bàng.
Thông tin ban đầu, khoảng 23h30 ngày 4-4, các trinh sát Phòng cảnh sát điều tra tội phạm về ma túy đột kích, phát hiện nhóm "bay lắc" gồm 12 dân chơi (7 nam, 5 nữ), thu giữ tại hiện trường 0,13g ketamin cùng một số tang vật khác có liên quan.
Kết quả giám định phát hiện 9 trường hợp dương tính với ma túy. Cơ quan điều tra sau đó tạm giữ hình sự ba trường hợp, gồm Nguyễn Thị Thanh Huyền (38 tuổi), Bùi Thị Ngọc Bích (36 tuổi) và Vũ Hoàng Cường (42 tuổi, cùng trú Hải Phòng) để điều tra về hành vi "tổ chức sử dụng trái phép chất ma túy".
Trong đó Bùi Thị Ngọc Bích là nữ cán bộ đang công tác tại Phòng cảnh sát phòng cháy chữa cháy Công an TP Hải Phòng.
Ngoài ra, còn một nữ cán bộ khác là V.A. có mặt tại buổi "bay lắc" là cán bộ đang công tác tại Công an quận Dương Kinh, TP Hải Phòng.

"""
gen_kwargs = {'length_penalty': 1, 'num_beams': 8,'max_length': 1024}


print(pipe(custom_dialogue, **gen_kwargs))

# EVALUATE

In [ ]:
import transformers
import evaluate
from transformers import pipeline
import pandas as pd
rouge = evaluate.load('rouge')
gen_sum=[]
hum_sum=[]
model_name="vibart_vietnews"
pipe = pipeline('summarization', model='./model/bart_vietnews_model')
gen_kwargs = {'length_penalty': 1, 'num_beams': 8,'max_length': 1024}

In [ ]:
from transformers import logging

logging.set_verbosity_error()
gen_sum=[]
for sect in dataset['validate']['Abstract']:
    gen = pipe(sect, **gen_kwargs)
    gen_sum.append(gen[0]['summary_text'])
    

In [ ]:
for title in dataset['validate']['Title']:
    hum_sum.append(title)

In [ ]:
results = rouge.compute(predictions=gen_sum,references=hum_sum)

In [ ]:
# Create a new DataFrame from the lists
import datetime
new_df = pd.DataFrame({
    'human': hum_sum,
    'generated': gen_sum
})
x = datetime.datetime.now()
time="_".join([model_name,x.strftime("%d"),x.strftime("%m"),x.strftime("%Y"),x.strftime("%H"),x.strftime("%M"),x.strftime("%S")])
# Save the new DataFrame to a CSV file
new_df.to_csv("".join(['summaries',time,'.csv']), index=False,encoding="utf_8_sig")
print("Complete: ",results)